## MMTHE01 - Masters Thesis

### E3. Thesis - Apply and Evaluate different XAI methods - Case Study with the ANN Model

* Applying XAI on a Deep Learning AI model (ANN Model)

#### Importing the libraries

In [ ]:
### import general libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import os
import time
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import recall_score
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE

from sklearn.preprocessing import LabelEncoder

In [ ]:
# Check your current working directory
cwd = os.getcwd()

# Define your relative path
relative_path = r"6. Analysis"  # adjust this relative to cwd

# Build the full path
full_path = os.path.join(cwd, relative_path)

# Check if it exists before changing
if os.path.exists(full_path):
    os.chdir(full_path)
    print("Changed directory to:", full_path)
else:
    print("Folder does not exist:", full_path)

#### Importing the dataset

In [ ]:
dataset = pd.read_csv('train_dataset_final_encoded.csv')

In [ ]:
dataset.head()

In [ ]:
dataset.shape

### 5.1 Split the data into Train-Test

#### 5.1.1 Separate the features and the label

In [ ]:
dataset_final = dataset.drop('TransactionID', axis=1)

In [ ]:
#X = dataset.iloc[:, 1:].values
#y = dataset.iloc[:,0].values

In [ ]:
X = dataset_final.iloc[:, 1:]
y = dataset_final.iloc[:,0]

In [ ]:
from sklearn.model_selection import train_test_split
X_train_im, X_test, y_train_im, y_test = train_test_split(X, y, test_size = 0.2, stratify=y, random_state = 1)

### 5.2 Applying SMOTE

In [ ]:
# Applying SMOTE only to the training data
smote = SMOTE(random_state=1)
X_train, y_train = smote.fit_resample(X_train_im, y_train_im)

### 5.3 Feature Scaling

In [ ]:
sc = StandardScaler()
X_tn_scaled = sc.fit_transform(X_train)
X_tt_scaled = sc.fit_transform(X_test)


# Convert to dataframe
X_train_scaled = pd.DataFrame(X_tn_scaled, columns=X_train.columns)
X_test_scaled = pd.DataFrame(X_tt_scaled, columns=X_test.columns)

### 5.3 Model Fitting

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
# Build an ANN model
model = Sequential([
    Input(shape=(X_train.shape[1],)),  # Explicit Input layer instead of input_dim in Dense
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dropout(0.2),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['AUC'])

# Early stopping to prevent overfitting
early_stop = EarlyStopping(monitor='val_AUC', patience=3, restore_best_weights=True, mode='max')

In [ ]:
# Train the ANN model with timing
history = model.fit(
    X_train_scaled, y_train,
    validation_split=0.2,
    epochs=20,
    batch_size=256,
    callbacks=[early_stop],
    verbose=1
)

### 5.5 Applying XAI methods to the ANN Model

#### 5.5.2 Applying LIME to the ANN Model

In [ ]:
import lime
import lime.lime_tabular

In [ ]:
X_test_scaled.head()

In [ ]:
start_time = time.time()

In [ ]:
lime_explainer = lime.lime_tabular.LimeTabularExplainer(
    training_data=X_train_scaled,
    feature_names=feature_names,
    mode='classification',
    verbose=True
)

In [ ]:
i = 0 # Choose an index to explain
sample = X_test_scaled.values[i]

In [ ]:
lime_exp = lime_explainer.explain_instance(
    data_row=X_test_scaled.values[i],
    predict_fn=model.predict_proba,
    num_features=10
)

In [ ]:
print(lime_exp.as_list())

In [ ]:
lime_exp.show_in_notebook()

In [ ]:
end_time = time.time()
training_time = end_time - start_time
print(f"LIME on ANN Model (Explanation Time): {training_time:.2f} seconds")

#### 5.5.3 Applying Submodular Pick (SP)-LIME to the ANN Model
* This is a variant of LIME

In [ ]:
import lime
import lime.lime_tabular
from lime import submodular_pick

In [ ]:
feature_names = X_train.columns.tolist()

In [ ]:
sample_size = 100
num_exps_desired = 5

In [ ]:
from contextlib import contextmanager
import sys, os

@contextmanager
def suppress_stdout_stderr():
    # Suppress stdout and stderr
    with open(os.devnull, "w") as devnull:
        old_stdout, old_stderr = sys.stdout, sys.stderr
        sys.stdout, sys.stderr = devnull, devnull
        try:
            yield
        finally:
            sys.stdout, sys.stderr = old_stdout, old_stderr

In [ ]:
start_time = time.time()

In [ ]:
lime_explainer = lime.lime_tabular.LimeTabularExplainer(
    training_data=X_train_scaled.values,
    feature_names=feature_names,
    class_names=["non-fraud", "fraud"],
    mode='classification',
    verbose=True,
    random_state = 1
)

In [ ]:
# Define the prediction function
predict_fn = lambda x: model.predict_proba(x).astype(float)

In [ ]:
# Initialize the SubmodularPick object
with suppress_stdout_stderr():
    sp_obj = submodular_pick.SubmodularPick(
        lime_explainer,
        X_train_scaled.values,
        predict_fn,
        sample_size=sample_size,
        num_features=10,
        num_exps_desired=num_exps_desired
    )

In [ ]:
end_time = time.time()
explanation_time = end_time - start_time
print(f"SP-LIME on ANN (Explanation Time): {explanation_time:.2f} seconds")

In [ ]:
selected_explanations = sp_obj.explanations[:num_exps_desired]

In [ ]:
for exp in selected_explanations:
    predicted_class = list(exp.local_exp.keys())[0]
    fig = exp.as_pyplot_figure(label=predicted_class)
    # Label the x-axis
    plt.xlabel("Feature Contribution")
    plt.show()

#### 5.5.4 Applying NormLIME to the ANN Model
* This is a variant of LIME

In [ ]:
from collections import Counter

In [ ]:
start_time = time.time()

In [ ]:
lime_explainer = lime.lime_tabular.LimeTabularExplainer(
    training_data=X_train_scaled.values,
    feature_names=feature_names,
    class_names=["non-fraud", "fraud"],
    mode='classification',
    verbose=True,
    random_state = 1
)

In [ ]:
# Generate explanations for multiple instances. Suppress textual output
with suppress_stdout_stderr():
    explanations = []
    for i in range(100):  # Adjust the number of instances as needed
        explanation = lime_explainer.explain_instance(X_test_scaled.iloc[i].values, model.predict_proba)
        explanations.append(explanation)

In [ ]:
# Aggregate feature importance
feature_importance = Counter()
for explanation in explanations:
    for feature, weight in explanation.as_list():
        feature_importance[feature] += weight

In [ ]:
# Normalize the feature importance
total_importance = sum(feature_importance.values())
normalized_importance = {feature: weight / total_importance for feature, weight in feature_importance.items()}

In [ ]:
end_time = time.time()
explanation_time = end_time - start_time
print(f"NormLIME on ANN (Explanation Time): {explanation_time:.2f} seconds")

In [ ]:
#Visualize the feature importance
# Sort features by importance
sorted_features = sorted(normalized_importance.items(), key=lambda x: x[1], reverse=True)

# Plot the top N features
top_n = 10
features, importances = zip(*sorted_features[:top_n])
plt.barh(features, importances)
plt.xlabel('Normalized Importance')
plt.title('Top {} Features by Importance'.format(top_n))
plt.show()